# Useful resources

http://stackoverflow.com/questions/9401658/matplotlib-animating-a-scatter-plot

https://plot.ly/python/streaming-bubbles-tutorial/

https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df=pd.read_pickle('../datasets/df_userID.pickle')

In [3]:
df['Age']=df.RaceYear-df.Year

In [4]:
df=df[df.Age>4]

In [5]:
df=df[df.Age<100]

In [6]:
categories=[0,5,10,15.5,25,43]

In [7]:
mens=df[df.Sex=='M']

In [8]:
df_categories=[df[(df.Distance>categories[i-1]) &(df.Distance<categories[i])] for i in range(1,len(categories))]

In [9]:
[x.shape[0] for x in df_categories]

[265021, 312078, 188205, 365711, 159084]

In [10]:
ages=list(range(4,81,4))

In [11]:
df_categories_age=[[] for i in df_categories]

In [12]:
for j in range(len(categories)-1):
    df_category=df_categories[j]
    df_categories_age[j]=[df_category[(df_category.Age>ages[i-1]) &(df_category.Age<ages[i])] for i in range(1,len(ages))]

In [91]:
df_categories_age_size=[[0 for i in range(len(df_categories_age[0]))] for i in range(len(df_categories_age))]

In [92]:
for i in range(len(df_categories_age)):
    for j in range(len(df_categories_age[0])):
        df_categories_age_size[i][j]=df_categories_age[i][j].shape[0]
df_categories_age_size=np.array(df_categories_age_size)

In [13]:
for i in df_categories_age[3]:
    print(i.shape[0])

5
105
784
5945
11224
20594
27413
30666
35750
40333
36520
27038
16894
9984
5796
2955
1370
469
133


In [93]:
paces_matrix=[[0 for j in range(len(ages)-1)] for i in range(len(categories)-1)]

In [94]:
for i in range(len(categories)-1):
    for j in range(len(ages)-1):
        paces_matrix[i][j]=np.mean(df_categories_age[i][j].Pace)

In [95]:
np_paces_matrix=np.array(paces_matrix)

In [96]:
np_paces=np.zeros(np_paces_matrix.shape)

In [97]:
for i in range(np_paces_matrix.shape[0]):
    for j in range(np_paces_matrix.shape[1]):
        np_paces[i,j]=np_paces_matrix[i,j].total_seconds()

In [98]:
def interpolate(matrix,interpolation=10):
    return_matrix=np.zeros([matrix.shape[0],(matrix.shape[1]-1)*interpolation+1])
    
    return_matrix[:,0]=matrix[:,0]
    starting_index=0
    for j in range(1,matrix.shape[1]):
        gradient=(matrix[:,j]-matrix[:,j-1])/interpolation
        for i in range(1,interpolation):
            return_matrix[:,starting_index+i]=return_matrix[:,starting_index]+i*gradient
        return_matrix[:,starting_index+interpolation]=matrix[:,j]
        starting_index+=interpolation
    return return_matrix

In [110]:
df_categories_age_size=interpolate(df_categories_age_size[:,1:])

In [111]:
np_paces=interpolate(np_paces[:,1:])

In [112]:
df_categories_age_size.shape

(5, 1691)

In [125]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=10, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        y,s,time_step = next(self.stream)
        y = [r for r in y]
        x = [1 for i in y]
#         s = [1000 for i in y]
        c = 1
#         print(y)
#         print(s)
        self.scat = self.ax.scatter(x, y, s=s, animated=True)
        self.ax.axis([-20, 20, 250, 500])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data=np_paces
        size=df_categories_age_size
        size=np.sqrt(size)*10
        
        for i in range(1,data.shape[1]):
            yield data[:,i],size[:,i],i
        
        #         data = np.random.random((4, self.numpoints))
#         xy = data[:2, :]
#         s, c = data[2:, :]
#         xy -= 0.5
#         xy *= 10
#         for i in range(1000):
#             xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
#             s += 0.05 * (np.random.random(self.numpoints) - 0.5)
#             c += 0.02 * (np.random.random(self.numpoints) - 0.5)
#             yield data

    def update(self, i):
        """Update the scatter plot."""
        try:
            y,s,time_step = next(self.stream)
        except:
            return self.scat
        y = [r for r in y]
        x = [1 for i in y]
#         s = [1000 for i in y]
        c = 1
#         print(y)
#         print(s)
        graph=np.row_stack((y,x))
        # Set x and y data...
        self.scat.set_offsets(graph)
        # Set sizes...
        self.scat._sizes = s# 300 * abs(data[2])**1.5 + 100

        # Set colors..
#         self.scat.set_array(data[3])
        self.scat =self.ax.scatter(x, y, s=s, animated=True)

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.show()


In [127]:
a = AnimatedScatter()
a.show()

# Old code

In [ ]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x, y, s, c = next(self.stream)
        self.scat = self.ax.scatter(x, y, c=c, s=s, animated=True)
        self.ax.axis([-10, 10, -10, 10])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data=np_paces_matrix
        
        for i in range(1,data.shape[1]):
            yield data[:,i]
        
        #         data = np.random.random((4, self.numpoints))
#         xy = data[:2, :]
#         s, c = data[2:, :]
#         xy -= 0.5
#         xy *= 10
#         for i in range(1000):
#             xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
#             s += 0.05 * (np.random.random(self.numpoints) - 0.5)
#             c += 0.02 * (np.random.random(self.numpoints) - 0.5)
#             yield data

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[:2, :])
        # Set sizes...
        self.scat._sizes = 300 * abs(data[2])**1.5 + 100
        # Set colors..
        self.scat.set_array(data[3])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.show()
